[View in Colab](https://colab.research.google.com/github/username/rice-breeding-simulation/blob/main/rice_breeding_simulation_mobps.ipynb)

# Rice Breeding Simulation Using MoBPS

This notebook simulates rice breeding programs with two varieties, linking genotypes and phenotypes through QTL for both production and climate resilience traits. We'll compare different selection methods (PBLUP, GBLUP, SSGBLUP) over 5 generations, and evaluate crossbreeding outcomes.

In [ ]:
# Install required packages
install.packages(c("MoBPS", "rrBLUP", "sommer", "Matrix", "methods", "stats", "utils"), repos="https://cran.rstudio.com/")

In [ ]:
# Load required libraries
library(MoBPS)
library(rrBLUP)
library(sommer)

## Step 1: Define simulation parameters

First, we'll define the parameters for our rice breeding simulation, including genetic architecture and trait characteristics.

In [ ]:
# Set random seed for reproducibility
set.seed(123)

# Simulation parameters
n_individuals_per_variety <- 200  # Number of individuals in each variety
n_snps <- 5000                    # Number of SNPs in the genome
n_qtls_production <- 50           # Number of QTLs for production trait
n_qtls_resilience <- 50           # Number of QTLs for climate resilience trait
n_generations <- 5                # Number of selection generations
heritability_production <- 0.5    # Heritability of production trait
heritability_resilience <- 0.4    # Heritability of climate resilience trait

# Selection intensities for different generations
selection_intensities <- c(0.1, 0.2, 0.3, 0.4, 0.5)  # Select top 10%, 20%, 30%, 40%, 50%

## Step 2: Create initial populations for two rice varieties

We'll generate two distinct rice varieties with different genetic backgrounds.

In [ ]:
# Create Variety 1 (initial population)
cat("Creating Variety 1...\n")
variety1_pop <- creating.diploid(
  nindi = n_individuals_per_variety,  # Number of individuals
  nsnp = n_snps,                      # Number of SNPs
  n.additive = c(n_qtls_production, n_qtls_resilience),  # QTLs for both traits
  heritability = c(heritability_production, heritability_resilience),
  mean.target = c(0, 0),             # Mean breeding values
  var.target = c(1, 1),              # Variance of breeding values
  name.cohort = "Variety1_Founders"
)

# Create Variety 2 (initial population)
cat("Creating Variety 2...\n")
variety2_pop <- creating.diploid(
  nindi = n_individuals_per_variety,  # Number of individuals
  nsnp = n_snps,                      # Number of SNPs
  n.additive = c(n_qtls_production, n_qtls_resilience),  # QTLs for both traits
  heritability = c(heritability_production, heritability_resilience),
  mean.target = c(0, 0),             # Mean breeding values
  var.target = c(1, 1),              # Variance of breeding values
  name.cohort = "Variety2_Founders"
)

cat("Initial populations created successfully!\n")
cat("Variety 1 - Number of individuals:", length(variety1_pop$breeding[[1]]$indi), "\n")
cat("Variety 2 - Number of individuals:", length(variety2_pop$breeding[[1]]$indi), "\n")

## Step 3: Link genotypes and phenotypes through QTL

We'll simulate phenotypes for both traits based on the underlying QTL architecture.

In [ ]:
# Generate phenotypes for Variety 1
cat("Generating phenotypes for Variety 1...\n")
variety1_pop <- breeding.diploid(
  variety1_pop,
  phenotyping.cohorts = "Variety1_Founders",
  heritability = c(heritability_production, heritability_resilience),
  n.observation = c(1, 1)  # One observation per individual for each trait
)

# Generate phenotypes for Variety 2
cat("Generating phenotypes for Variety 2...\n")
variety2_pop <- breeding.diploid(
  variety2_pop,
  phenotyping.cohorts = "Variety2_Founders",
  heritability = c(heritability_production, heritability_resilience),
  n.observation = c(1, 1)  # One observation per individual for each trait
)

cat("Phenotypes generated successfully!\n")

# Extract and display some phenotype data
pheno_var1 <- get.pheno(variety1_pop, cohorts = "Variety1_Founders")
pheno_var2 <- get.pheno(variety2_pop, cohorts = "Variety2_Founders")

cat("Variety 1 - Production trait stats: Mean = ", round(mean(pheno_var1[,1], na.rm=TRUE), 2), 
    ", SD = ", round(sd(pheno_var1[,1], na.rm=TRUE), 2), "\n")
cat("Variety 1 - Climate resilience trait stats: Mean = ", round(mean(pheno_var1[,2], na.rm=TRUE), 2), 
    ", SD = ", round(sd(pheno_var1[,2], na.rm=TRUE), 2), "\n")
cat("Variety 2 - Production trait stats: Mean = ", round(mean(pheno_var2[,1], na.rm=TRUE), 2), 
    ", SD = ", round(sd(pheno_var2[,1], na.rm=TRUE), 2), "\n")
cat("Variety 2 - Climate resilience trait stats: Mean = ", round(mean(pheno_var2[,2], na.rm=TRUE), 2), 
    ", SD = ", round(sd(pheno_var2[,2], na.rm=TRUE), 2), "\n")

## Step 4: Implement selection methods comparison (PBLUP, GBLUP, SSGBLUP)

We'll run selection experiments with different methods for each variety over 5 generations.

In [ ]:
# Initialize results containers
results_pblup_var1 <- list()
results_gblup_var1 <- list()
results_ssgblup_var1 <- list()
results_pblup_var2 <- list()
results_gblup_var2 <- list()
results_ssgblup_var2 <- list()

# Function to run selection for a specific method
run_selection_method <- function(population, method_name, generations, intensities) {
  pop <- population
  results <- list()
  
  for(gen in 1:generations) {
    cat("Generation", gen, "using", method_name, "method...\n")
    
    # Calculate breeding values using specified method
    if(method_name == "PBLUP") {
      pop <- breeding.diploid(
        pop,
        bve = TRUE,
        bve.cohorts = paste0("Gen", gen-1, if(gen == 1) "_Founders" else ""),
        relationship.matrix = "kinship",  # Pedigree-based relationship matrix
        bve.gen = length(pop$breeding)
      )
    } else if(method_name == "GBLUP") {
      pop <- breeding.diploid(
        pop,
        bve = TRUE,
        bve.cohorts = paste0("Gen", gen-1, if(gen == 1) "_Founders" else ""),
        relationship.matrix = "GBLUP",  # Genomic relationship matrix
        bve.gen = length(pop$breeding)
      )
    } else if(method_name == "SSGBLUP") {
      pop <- breeding.diploid(
        pop,
        bve = TRUE,
        bve.cohorts = paste0("Gen", gen-1, if(gen == 1) "_Founders" else ""),
        relationship.matrix = "GBLUP",
        singlestep.active = TRUE,  # Activate single-step approach
        bve.gen = length(pop$breeding)
      )
    }
    
    # Determine number of individuals to select based on intensity
    n_select <- max(1, floor(intensities[gen] * n_individuals_per_variety))
    
    # Perform selection
    pop <- breeding.diploid(
      pop,
      breeding.size = n_individuals_per_variety,  # Maintain population size
      selection.size = c(n_select, n_select),     # Select top individuals for breeding
      selection.criteria = "bve",                 # Use estimated breeding values
      selection.m.cohorts = paste0("Gen", gen-1, if(gen == 1) "_Founders" else ""),
      selection.f.cohorts = paste0("Gen", gen-1, if(gen == 1) "_Founders" else ""),
      name.cohort = paste0("Gen", gen),
      add.gen = length(pop$breeding) + 1
    )
    
    # Store results for this generation
    gen_bvs <- get.bv(pop, gen = length(pop$breeding))
    gen_phenos <- get.pheno(pop, gen = length(pop$breeding))
    
    results[[gen]] <- list(
      generation = gen,
      mean_bv_prod = mean(gen_bvs[,1]),
      mean_bv_res = mean(gen_bvs[,2]),
      mean_pheno_prod = mean(gen_phenos[,1], na.rm=TRUE),
      mean_pheno_res = mean(gen_phenos[,2], na.rm=TRUE),
      sd_bv_prod = sd(gen_bvs[,1]),
      sd_bv_res = sd(gen_bvs[,2]),
      accuracy_prod = cor(gen_bvs[,1], gen_phenos[,1], use="complete.obs"),
      accuracy_res = cor(gen_bvs[,2], gen_phenos[,2], use="complete.obs")
    )
    
    cat("  Accuracy for production trait:", round(results[[gen]]$accuracy_prod, 3), "\n")
    cat("  Accuracy for resilience trait:", round(results[[gen]]$accuracy_res, 3), "\n")
  }
  
  return(list(population = pop, results = results))
}

# Run selection for Variety 1
cat("\nStarting selection for Variety 1:\n")
cat("Running PBLUP...\n")
results_pblup_var1_out <- run_selection_method(variety1_pop, "PBLUP", n_generations, selection_intensities)
results_pblup_var1 <- results_pblup_var1_out$results

cat("\nRunning GBLUP...\n")
results_gblup_var1_out <- run_selection_method(variety1_pop, "GBLUP", n_generations, selection_intensities)
results_gblup_var1 <- results_gblup_var1_out$results

cat("\nRunning SSGBLUP...\n")
results_ssgblup_var1_out <- run_selection_method(variety1_pop, "SSGBLUP", n_generations, selection_intensities)
results_ssgblup_var1 <- results_ssgblup_var1_out$results

# Run selection for Variety 2
cat("\nStarting selection for Variety 2:\n")
cat("Running PBLUP...\n")
results_pblup_var2_out <- run_selection_method(variety2_pop, "PBLUP", n_generations, selection_intensities)
results_pblup_var2 <- results_pblup_var2_out$results

cat("\nRunning GBLUP...\n")
results_gblup_var2_out <- run_selection_method(variety2_pop, "GBLUP", n_generations, selection_intensities)
results_gblup_var2 <- results_gblup_var2_out$results

cat("\nRunning SSGBLUP...\n")
results_ssgblup_var2_out <- run_selection_method(variety2_pop, "SSGBLUP", n_generations, selection_intensities)
results_ssgblup_var2 <- results_ssgblup_var2_out$results

## Step 5: Calculate inbreeding coefficients

We'll measure inbreeding levels across generations to assess genetic diversity.

In [ ]:
# Function to calculate inbreeding coefficients
calculate_inbreeding <- function(population, generations) {
  inbreeding_results <- list()
  
  for(gen in 1:generations) {
    # Calculate empirical inbreeding coefficients
    inbreeding_vals <- inbreeding.emp(population, gen = gen)
    
    inbreeding_results[[gen]] <- list(
      generation = gen,
      mean_inbreeding = mean(inbreeding_vals, na.rm=TRUE),
      max_inbreeding = max(inbreeding_vals, na.rm=TRUE),
      sd_inbreeding = sd(inbreeding_vals, na.rm=TRUE)
    )
  }
  
  return(inbreeding_results)
}

# Calculate inbreeding for all methods and varieties
inbreeding_pblup_var1 <- calculate_inbreeding(results_pblup_var1_out$population, n_generations)
inbreeding_gblup_var1 <- calculate_inbreeding(results_gblup_var1_out$population, n_generations)
inbreeding_ssgblup_var1 <- calculate_inbreeding(results_ssgblup_var1_out$population, n_generations)

inbreeding_pblup_var2 <- calculate_inbreeding(results_pblup_var2_out$population, n_generations)
inbreeding_gblup_var2 <- calculate_inbreeding(results_gblup_var2_out$population, n_generations)
inbreeding_ssgblup_var2 <- calculate_inbreeding(results_ssgblup_var2_out$population, n_generations)

## Step 6: Plot response to selection and inbreeding coefficients

Visualize the genetic gain and inbreeding trends across selection methods.

In [ ]:
# Load plotting library
library(ggplot2)

# Prepare data for plotting
plot_data <- data.frame(
  Generation = rep(1:n_generations, 6),
  Method = c(
    rep("PBLUP_V1", n_generations),
    rep("GBLUP_V1", n_generations),
    rep("SSGBLUP_V1", n_generations),
    rep("PBLUP_V2", n_generations),
    rep("GBLUP_V2", n_generations),
    rep("SSGBLUP_V2", n_generations)
  ),
  Mean_BV_Production = c(
    sapply(results_pblup_var1, function(x) x$mean_bv_prod),
    sapply(results_gblup_var1, function(x) x$mean_bv_prod),
    sapply(results_ssgblup_var1, function(x) x$mean_bv_prod),
    sapply(results_pblup_var2, function(x) x$mean_bv_prod),
    sapply(results_gblup_var2, function(x) x$mean_bv_prod),
    sapply(results_ssgblup_var2, function(x) x$mean_bv_prod)
  ),
  Mean_Inbreeding = c(
    sapply(inbreeding_pblup_var1, function(x) x$mean_inbreeding),
    sapply(inbreeding_gblup_var1, function(x) x$mean_inbreeding),
    sapply(inbreeding_ssgblup_var1, function(x) x$mean_inbreeding),
    sapply(inbreeding_pblup_var2, function(x) x$mean_inbreeding),
    sapply(inbreeding_gblup_var2, function(x) x$mean_inbreeding),
    sapply(inbreeding_ssgblup_var2, function(x) x$mean_inbreeding)
  ),
  Variety = c(
    rep("Variety 1", n_generations*3),
    rep("Variety 2", n_generations*3)
  )
)

# Plot response to selection (mean breeding values for production trait)
p1 <- ggplot(plot_data, aes(x = Generation, y = Mean_BV_Production, color = Method, linetype = Variety)) +
  geom_line(size = 1) +
  geom_point(size = 2) +
  labs(title = "Response to Selection - Mean Breeding Values for Production Trait",
       x = "Generation",
       y = "Mean Breeding Value") +
  theme_minimal() +
  scale_color_brewer(type = "qual", palette = "Set1")

# Plot inbreeding coefficients
p2 <- ggplot(plot_data, aes(x = Generation, y = Mean_Inbreeding, color = Method, linetype = Variety)) +
  geom_line(size = 1) +
  geom_point(size = 2) +
  labs(title = "Inbreeding Coefficients Across Generations",
       x = "Generation",
       y = "Mean Inbreeding Coefficient") +
  theme_minimal() +
  scale_color_brewer(type = "qual", palette = "Set1")

# Display plots
print(p1)
print(p2)

## Step 7: Crossbreeding Experiment and Heterosis Calculation

Perform crossbreeding between the two varieties and calculate heterosis.

In [ ]:
# Function to perform crossbreeding and calculate heterosis
perform_crossbreeding <- function(parent_var1_pop, parent_var2_pop, n_crosses = 100) {
  # Extract the final populations after selection
  # We'll use the final generation from GBLUP results for both varieties
  
  # Get the last generation from the GBLUP results for both varieties
  gen_count_var1 <- length(parent_var1_pop$breeding)
  gen_count_var2 <- length(parent_var2_pop$breeding)
  
  # Create a crossbred population
  # For simplicity, we'll mate individuals from variety 1 (as sires) with variety 2 (as dams)
  
  # Create crossbred offspring
  crossbred_pop <- breeding.diploid(
    parent_var1_pop,  # Start with variety 1 as base
    breeding.size = n_crosses,
    selection.size = c(min(50, n_individuals_per_variety), min(50, n_individuals_per_variety)),  # Select 50 parents from each variety
    selection.m.cohorts = paste0("Gen", n_generations),  # Use the final generation
    selection.f.cohorts = paste0("Gen", n_generations),  # Use the final generation
    name.cohort = "Crossbred_F1",
    add.gen = gen_count_var1 + 1
  )
  
  # Generate phenotypes for crossbred population
  crossbred_pop <- breeding.diploid(
    crossbred_pop,
    phenotyping.cohorts = "Crossbred_F1",
    heritability = c(heritability_production, heritability_resilience),
    n.observation = c(1, 1)
  )
  
  # Calculate heterosis
  # Get phenotypes for parents and crossbred offspring
  pheno_crossbred <- get.pheno(crossbred_pop, cohorts = "Crossbred_F1")
  
  # For simplicity, we'll assume we have the parental phenotypes from the last generation
  # In practice, we would need to track the specific parents used
  
  # Calculate mid-parent values
  # Since we don't have the exact parental values, we'll approximate using the population means
  
  # Calculate heterosis as: H = F1 - MP
  # Where F1 is the crossbred offspring value and MP is the mid-parent value
  
  # Calculate heterosis for each trait
  mean_crossbred_prod <- mean(pheno_crossbred[,1], na.rm=TRUE)
  mean_crossbred_res <- mean(pheno_crossbred[,2], na.rm=TRUE)
  
  # For demonstration, we'll use the overall population means as proxies for mid-parent values
  # This is a simplification - in practice you'd use the actual parental values
  mean_parental_prod <- (results_gblup_var1[[n_generations]]$mean_pheno_prod + 
                         results_gblup_var2[[n_generations]]$mean_pheno_prod) / 2
  mean_parental_res <- (results_gblup_var2[[n_generations]]$mean_pheno_res + 
                        results_gblup_var2[[n_generations]]$mean_pheno_res) / 2
  
  heterosis_prod <- mean_crossbred_prod - mean_parental_prod
  heterosis_res <- mean_crossbred_res - mean_parental_res
  
  # Calculate heterosis as percentage
  heterosis_pct_prod <- (heterosis_prod / mean_parental_prod) * 100
  heterosis_pct_res <- (heterosis_res / mean_parental_res) * 100
  
  return(list(
    crossbred_population = crossbred_pop,
    mean_crossbred_prod = mean_crossbred_prod,
    mean_crossbred_res = mean_crossbred_res,
    mean_parental_prod = mean_parental_prod,
    mean_parental_res = mean_parental_res,
    heterosis_abs_prod = heterosis_prod,
    heterosis_abs_res = heterosis_res,
    heterosis_pct_prod = heterosis_pct_prod,
    heterosis_pct_res = heterosis_pct_res
  ))
}

# Perform crossbreeding with the GBLUP-selected populations
crossbreeding_results <- perform_crossbreeding(
  results_gblup_var1_out$population, 
  results_gblup_var2_out$population
)

# Print heterosis results
cat("Crossbreeding Results:\n")
cat("Mean production trait in crossbreds:", round(crossbreeding_results$mean_crossbred_prod, 3), "\n")
cat("Mean production trait in parents:", round(crossbreeding_results$mean_parental_prod, 3), "\n")
cat("Absolute heterosis for production:", round(crossbreeding_results$heterosis_abs_prod, 3), "\n")
cat("Percentage heterosis for production:", round(crossbreeding_results$heterosis_pct_prod, 2), "%\n")
cat("\n")
cat("Mean resilience trait in crossbreds:", round(crossbreeding_results$mean_crossbred_res, 3), "\n")
cat("Mean resilience trait in parents:", round(crossbreeding_results$mean_parental_res, 3), "\n")
cat("Absolute heterosis for resilience:", round(crossbreeding_results$heterosis_abs_res, 3), "\n")
cat("Percentage heterosis for resilience:", round(crossbreeding_results$heterosis_pct_res, 2), "%\n")

## Step 8: Summary of Results and Comparison of Methods

Compare the performance of different selection methods across all metrics.

In [ ]:
# Create summary table for comparison
summary_table <- data.frame(
  Method = c("PBLUP_V1", "GBLUP_V1", "SSGBLUP_V1", "PBLUP_V2", "GBLUP_V2", "SSGBLUP_V2"),
  Final_Mean_BV_Prod = c(
    results_pblup_var1[[n_generations]]$mean_bv_prod,
    results_gblup_var1[[n_generations]]$mean_bv_prod,
    results_ssgblup_var1[[n_generations]]$mean_bv_prod,
    results_pblup_var2[[n_generations]]$mean_bv_prod,
    results_gblup_var2[[n_generations]]$mean_bv_prod,
    results_ssgblup_var2[[n_generations]]$mean_bv_prod
  ),
  Final_Mean_BV_Res = c(
    results_pblup_var1[[n_generations]]$mean_bv_res,
    results_gblup_var1[[n_generations]]$mean_bv_res,
    results_ssgblup_var1[[n_generations]]$mean_bv_res,
    results_pblup_var2[[n_generations]]$mean_bv_res,
    results_gblup_var2[[n_generations]]$mean_bv_res,
    results_ssgblup_var2[[n_generations]]$mean_bv_res
  ),
  Final_Accuracy_Prod = c(
    results_pblup_var1[[n_generations]]$accuracy_prod,
    results_gblup_var1[[n_generations]]$accuracy_prod,
    results_ssgblup_var1[[n_generations]]$accuracy_prod,
    results_pblup_var2[[n_generations]]$accuracy_prod,
    results_gblup_var2[[n_generations]]$accuracy_prod,
    results_ssgblup_var2[[n_generations]]$accuracy_prod
  ),
  Final_Accuracy_Res = c(
    results_pblup_var1[[n_generations]]$accuracy_res,
    results_gblup_var1[[n_generations]]$accuracy_res,
    results_ssgblup_var1[[n_generations]]$accuracy_res,
    results_pblup_var2[[n_generations]]$accuracy_res,
    results_gblup_var2[[n_generations]]$accuracy_res,
    results_ssgblup_var2[[n_generations]]$accuracy_res
  ),
  Final_Inbreeding = c(
    inbreeding_pblup_var1[[n_generations]]$mean_inbreeding,
    inbreeding_gblup_var1[[n_generations]]$mean_inbreeding,
    inbreeding_ssgblup_var1[[n_generations]]$mean_inbreeding,
    inbreeding_pblup_var2[[n_generations]]$mean_inbreeding,
    inbreeding_gblup_var2[[n_generations]]$mean_inbreeding,
    inbreeding_ssgblup_var2[[n_generations]]$mean_inbreeding
  )
)

# Print summary table
print("Summary of Selection Methods Performance:")
print(summary_table)

# Identify best method for each variety based on final breeding values
cat("\nBest method for Variety 1 (Production trait): ", 
    summary_table[1:3,][which.max(summary_table[1:3,]$Final_Mean_BV_Prod), "Method"], "\n")
cat("Best method for Variety 1 (Resilience trait): ", 
    summary_table[1:3,][which.max(summary_table[1:3,]$Final_Mean_BV_Res), "Method"], "\n")
cat("Best method for Variety 2 (Production trait): ", 
    summary_table[4:6,][which.max(summary_table[4:6,]$Final_Mean_BV_Prod), "Method"], "\n")
cat("Best method for Variety 2 (Resilience trait): ", 
    summary_table[4:6,][which.max(summary_table[4:6,]$Final_Mean_BV_Res), "Method"], "\n")

## Step 9: Download Results

Prepare the results for download as CSV files.

In [ ]:
# Create data frames for results export
selection_results_df <- data.frame(
  Generation = rep(1:n_generations, 6),
  Method = c(rep("PBLUP_V1", n_generations), rep("GBLUP_V1", n_generations), 
            rep("SSGBLUP_V1", n_generations), rep("PBLUP_V2", n_generations),
            rep("GBLUP_V2", n_generations), rep("SSGBLUP_V2", n_generations)),
  Variety = c(rep("Variety 1", n_generations*3), rep("Variety 2", n_generations*3)),
  Mean_BV_Production = c(
    sapply(results_pblup_var1, function(x) x$mean_bv_prod),
    sapply(results_gblup_var1, function(x) x$mean_bv_prod),
    sapply(results_ssgblup_var1, function(x) x$mean_bv_prod),
    sapply(results_pblup_var2, function(x) x$mean_bv_prod),
    sapply(results_gblup_var2, function(x) x$mean_bv_prod),
    sapply(results_ssgblup_var2, function(x) x$mean_bv_prod)
  ),
  Mean_BV_Resilience = c(
    sapply(results_pblup_var1, function(x) x$mean_bv_res),
    sapply(results_gblup_var1, function(x) x$mean_bv_res),
    sapply(results_ssgblup_var1, function(x) x$mean_bv_res),
    sapply(results_pblup_var2, function(x) x$mean_bv_res),
    sapply(results_gblup_var2, function(x) x$mean_bv_res),
    sapply(results_ssgblup_var2, function(x) x$mean_bv_res)
  ),
  Accuracy_Production = c(
    sapply(results_pblup_var1, function(x) x$accuracy_prod),
    sapply(results_gblup_var1, function(x) x$accuracy_prod),
    sapply(results_ssgblup_var1, function(x) x$accuracy_prod),
    sapply(results_pblup_var2, function(x) x$accuracy_prod),
    sapply(results_gblup_var2, function(x) x$accuracy_prod),
    sapply(results_ssgblup_var2, function(x) x$accuracy_prod)
  ),
  Accuracy_Resilience = c(
    sapply(results_pblup_var1, function(x) x$accuracy_res),
    sapply(results_gblup_var1, function(x) x$accuracy_res),
    sapply(results_ssgblup_var1, function(x) x$accuracy_res),
    sapply(results_pblup_var2, function(x) x$accuracy_res),
    sapply(results_gblup_var2, function(x) x$accuracy_res),
    sapply(results_ssgblup_var2, function(x) x$accuracy_res)
  ),
  Mean_Inbreeding = c(
    sapply(inbreeding_pblup_var1, function(x) x$mean_inbreeding),
    sapply(inbreeding_gblup_var1, function(x) x$mean_inbreeding),
    sapply(inbreeding_ssgblup_var1, function(x) x$mean_inbreeding),
    sapply(inbreeding_pblup_var2, function(x) x$mean_inbreeding),
    sapply(inbreeding_gblup_var2, function(x) x$mean_inbreeding),
    sapply(inbreeding_ssgblup_var2, function(x) x$mean_inbreeding)
  )
)

# Save results to CSV files
write.csv(selection_results_df, "selection_results.csv", row.names = FALSE)
write.csv(summary_table, "summary_comparison.csv", row.names = FALSE)

# Create heterosis results dataframe
heterosis_df <- data.frame(
  Trait = c("Production", "Climate Resilience"),
  Crossbred_Mean = c(crossbreeding_results$mean_crossbred_prod, crossbreeding_results$mean_crossbred_res),
  Parental_Mean = c(crossbreeding_results$mean_parental_prod, crossbreeding_results$mean_parental_res),
  Absolute_Heterosis = c(crossbreeding_results$heterosis_abs_prod, crossbreeding_results$heterosis_abs_res),
  Percentage_Heterosis = c(crossbreeding_results$heterosis_pct_prod, crossbreeding_results$heterosis_pct_res)
)

write.csv(heterosis_df, "heterosis_results.csv", row.names = FALSE)

cat("Results saved to CSV files: selection_results.csv, summary_comparison.csv, heterosis_results.csv\n")

# In Google Colab, uncomment the following lines to download files:
# from google.colab import files
# files.download('selection_results.csv')
# files.download('summary_comparison.csv')
# files.download('heterosis_results.csv')

## Conclusion

This simulation demonstrates the implementation of rice breeding programs using MoBPS, comparing different selection methods (PBLUP, GBLUP, SSGBLUP) across two varieties. The simulation evaluates:

1. Genetic gain over 5 generations of selection
2. Accuracy of breeding value predictions
3. Inbreeding accumulation
4. Crossbreeding performance and heterosis

The results provide insights into which selection methods are most effective for improving production and climate resilience traits while managing inbreeding.